In [102]:
import numpy as np
import pandas as pd
from rdkit import Chem
from rdkit.Chem import PandasTools
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Embedding, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [103]:
sdf_data = '/content/3A52D1DC61F41351E72E780DF69DAEEBki.sdf'
df = PandasTools.LoadSDF(sdf_data, smilesName='SMILES', molColName='Molecule', includeFingerprints=True)

In [104]:
df['SMILES'] = df['Molecule'].apply(Chem.MolToSmiles)
tokenizer = Tokenizer(char_level=True)
tokenizer.fit_on_texts(df['SMILES'])
vocab_size = len(tokenizer.word_index) + 1
print("Vocab size:", vocab_size)

Vocab size: 26


In [105]:
sequences = tokenizer.texts_to_sequences(df['SMILES'])
max_length = max([len(seq) for seq in sequences]) + 1
X = []
y = []
for seq in sequences:
    for i in range(1, len(seq)):
        input_seq = seq[:i]
        target_seq = seq[i]
        X.append(input_seq)
        y.append(target_seq)

In [106]:
X = pad_sequences(X, maxlen=max_length, padding='post')
y = np.array(y)
y = np.expand_dims(y, -1)

print("X shape:", X.shape)
print("y shape:", y.shape)

X shape: (2041, 69)
y shape: (2041, 1)


In [112]:
model = Sequential()
model.add(Embedding(vocab_size, 50, input_length=max_length))
model.add(LSTM(128, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(128, return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(vocab_size, activation='softmax'))
model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam(learning_rate=0.01), metrics=['accuracy'])
model.fit(X, y, epochs=250, batch_size=32, validation_split=0.2)

Epoch 1/250
51/51 [==============================] - 5s 44ms/step - loss: 2.2099 - accuracy: 0.4553 - val_loss: 2.0219 - val_accuracy: 0.5012
Epoch 2/250
51/51 [==============================] - 1s 13ms/step - loss: 2.0829 - accuracy: 0.4663 - val_loss: 2.0397 - val_accuracy: 0.5012
Epoch 3/250
51/51 [==============================] - 1s 21ms/step - loss: 2.0830 - accuracy: 0.4663 - val_loss: 2.0437 - val_accuracy: 0.5012
Epoch 4/250
51/51 [==============================] - 1s 19ms/step - loss: 2.0702 - accuracy: 0.4663 - val_loss: 2.0350 - val_accuracy: 0.5012
Epoch 5/250
51/51 [==============================] - 1s 13ms/step - loss: 2.0616 - accuracy: 0.4663 - val_loss: 2.0424 - val_accuracy: 0.5012
Epoch 6/250
51/51 [==============================] - 1s 12ms/step - loss: 2.0715 - accuracy: 0.4663 - val_loss: 2.0471 - val_accuracy: 0.5012
Epoch 7/250
51/51 [==============================] - 1s 11ms/step - loss: 2.0787 - accuracy: 0.4663 - val_loss: 2.0580 - val_accuracy: 0.5012
Epoch 

In [113]:
def generate_SMILES(seed_text, next_words, model, max_seq_len, tokenizer):
    result = seed_text
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([result])[0]
        token_list = pad_sequences([token_list], maxlen=max_seq_len, padding='post')
        predicted_probs = model.predict(token_list, verbose=0)
        predicted_index = np.argmax(predicted_probs[0])
        print(f"Current result:{result}")
        print(f"Token list:{token_list}")
        print(f"Predicted probabilities:{predicted_probs}")
        print(f"Predicted index:{predicted_index}")
        if predicted_index in tokenizer.index_word:
            output_word = tokenizer.index_word[predicted_index]
        else:
            print(f"{predicted_index} is out of range.")
            break
        print(f"Output word: {output_word}")
        result+=output_word
        if output_word == '':
            break
    return result

seed_text = "N2NN"
new_smiles = generate_SMILES(seed_text, 50, model, max_length, tokenizer)
print("Generated SMILES: ", new_smiles)

Current result: N2NN
Token list: [[7 6 7 7 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]]
Predicted probabilities: [[1.01920996e-07 3.93027306e-01 7.13117570e-02 1.35093313e-02
  1.27422899e-01 4.30114829e-04 1.15020362e-06 1.31191373e-01
  1.48922140e-07 2.15366632e-02 4.07276758e-07 3.51871876e-03
  1.47451449e-03 6.84815968e-05 2.31630679e-06 8.17681570e-03
  5.53296832e-06 1.01711075e-05 1.10764377e-05 2.28295624e-01
  3.29683621e-06 3.35151242e-07 3.28757679e-08 5.05838438e-09
  1.68814415e-06 1.97019602e-07]]
Predicted index: 1
Output word: c
Current result: N2NNc
Token list: [[7 6 7 7 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]]
Predicted probabilities: [[8.71389219e-08 3.84256929e-01 1.13404155e-01 1.02562392e-02
  2.35505477e-01 1.29020796e-03 8.44649985e-05 1.69902876e-01
  1.50313849e-06 2.42552608e-02